In [21]:
!pip install paddlepaddle
!pip install paddlepaddle-gpu
!pip install paddleocr
import paddleocr
!git clone
from paddleocr import PaddleOCR, draw_ocr # main OCR dependencies
import cv2 #opencv
import os # folder directory navigation

ocr_model = PaddleOCR(lang='en')
img_path ="/cognizant.jpg"

# Perform OCR on the image
result = ocr_model.ocr(img_path)

# Print the entire OCR result
for line in result:
    print(' '.join([word_info[1][0] for word_info in line]))
    import os

# Define the output directory
output_dir = "C:/Users/Vishal/Desktop/out_dir"

# Check if the output directory exists, if not create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define the path for the output text file
output_text_file = os.path.join(output_dir, 'ocr_output.txt')

# Extract text from OCR result
ocr_text = ""
for line in result:
    ocr_text += ' '.join([word_info[1][0] for word_info in line]) + '\n'

# Write the extracted text into a text file
with open(output_text_file, 'w') as f:
    f.write(ocr_text)

print("OCR output has been written to:", output_text_file)

fatal: You must specify a repository to clone.

usage: git clone [<options>] [--] <repo> [<dir>]

    -v, --verbose         be more verbose
    -q, --quiet           be more quiet
    --progress            force progress reporting
    --reject-shallow      don't clone shallow repository
    -n, --no-checkout     don't create a checkout
    --bare                create a bare repository
    --mirror              create a mirror repository (implies bare)
    -l, --local           to clone from a local repository
    --no-hardlinks        don't use local hardlinks, always copy
    -s, --shared          setup as shared repository
    --recurse-submodules[=<pathspec>]
                          initialize submodules in the clone
    --recursive ...       alias of --recurse-submodules
    -j, --jobs <n>        number of submodules cloned in parallel
    --template <template-directory>
                          directory from which templates will be used
    --reference <repo>    reference rep

In [2]:
import pandas as pd
import spacy
from spacy.training import Example
import random

# Load spaCy model for organization names
nlp_org = spacy.blank("en")

# Define the NER pipeline component for organization names
ner_org = nlp_org.add_pipe("ner")
ner_org.add_label("ORG")

# Read data from CSV file for organization names
df_org = pd.read_csv("/content/Company Names.csv")  # Update with your CSV file

# Convert data into spaCy training format for organization names
train_examples_org = []
for _, row in df_org.iterrows():
    # Use organization name as the text
    text = row["Company Name"]  # Update with the correct column name
    # Add label for the entire text
    annotations = {"entities": [(0, len(text), "ORG")]}
    example = Example.from_dict(nlp_org.make_doc(text), annotations)
    train_examples_org.append(example)

# Training loop for organization names
other_pipes_org = [pipe for pipe in nlp_org.pipe_names if pipe != "ner"]
with nlp_org.disable_pipes(*other_pipes_org):
    optimizer_org = nlp_org.begin_training()
    for iteration in range(50):
        losses_org = {}
        random.shuffle(train_examples_org)
        for example in train_examples_org:
            nlp_org.update([example], drop=0.5, losses=losses_org)
        print("Organization Names - Iteration", iteration, "Losses", losses_org)

# Save the trained model for organization names
nlp_org.to_disk("ner_model_org")


Organization Names - Iteration 0 Losses {'ner': 596.8187235730832}
Organization Names - Iteration 1 Losses {'ner': 42.83886153970551}
Organization Names - Iteration 2 Losses {'ner': 19.120286726433736}
Organization Names - Iteration 3 Losses {'ner': 5.5143368921603395}
Organization Names - Iteration 4 Losses {'ner': 31.263607005621896}
Organization Names - Iteration 5 Losses {'ner': 9.788759623909136e-09}
Organization Names - Iteration 6 Losses {'ner': 8.785343626350159}
Organization Names - Iteration 7 Losses {'ner': 23.10561631982133}
Organization Names - Iteration 8 Losses {'ner': 9.95975898797382}
Organization Names - Iteration 9 Losses {'ner': 13.96748114247724}
Organization Names - Iteration 10 Losses {'ner': 16.058507174906264}
Organization Names - Iteration 11 Losses {'ner': 15.10430126668428}
Organization Names - Iteration 12 Losses {'ner': 6.996585023794156}
Organization Names - Iteration 13 Losses {'ner': 9.433614391593765e-09}
Organization Names - Iteration 14 Losses {'ner'

In [35]:
# Save the trained model for organization names
model_path = "/content/ner_model_org"
nlp_org.to_disk(model_path)

print("Model saved to:", model_path)


Model saved to: /content/ner_model_org


In [36]:
# Zip the model directory
!zip -r /content/ner_model_org.zip /content/ner_model_org


  adding: content/ner_model_org/ (stored 0%)
  adding: content/ner_model_org/meta.json (deflated 49%)
  adding: content/ner_model_org/ner/ (stored 0%)
  adding: content/ner_model_org/ner/moves (deflated 38%)
  adding: content/ner_model_org/ner/cfg (deflated 33%)
  adding: content/ner_model_org/ner/model (deflated 8%)
  adding: content/ner_model_org/tokenizer (deflated 81%)
  adding: content/ner_model_org/config.cfg (deflated 59%)
  adding: content/ner_model_org/vocab/ (stored 0%)
  adding: content/ner_model_org/vocab/vectors.cfg (stored 0%)
  adding: content/ner_model_org/vocab/strings.json (deflated 72%)
  adding: content/ner_model_org/vocab/vectors (deflated 45%)
  adding: content/ner_model_org/vocab/key2row (stored 0%)
  adding: content/ner_model_org/vocab/lookups.bin (stored 0%)


In [37]:
from google.colab import files

# Download the zipped model directory
files.download("/content/ner_model_org.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
import os
import re

def extract_organization_and_person_names(text):
    # Extract company name using regex (assuming it appears in the email domain)
    organization_match = re.search(r'@([A-Za-z]+)\.com', text)
    organization_name = organization_match.group(1).capitalize() if organization_match else ""

    # Extract person name using regex (assuming it appears before the email)
    person_name_match = re.search(r'([A-Za-z\.]+)@', text)
    person_name = person_name_match.group(1).replace('.', ' ').strip().title() if person_name_match else ""

    return organization_name, person_name

# Take input from the user
text = input("Enter the text for extraction: ")

# Extract company name and person name
organization_name, person_name = extract_organization_and_person_names(text)

# Specify the directories where you want to save the files
organization_dir = "/var/organization"
person_dir = "/var/person"

# Ensure directories exist, create if not
os.makedirs(organization_dir, exist_ok=True)
os.makedirs(person_dir, exist_ok=True)

# Write the extracted organization name to a text file
organization_output_filename = "organization_name.txt"
organization_output_path = os.path.join(organization_dir, organization_output_filename)
with open(organization_output_path, "w") as file:
    file.write(organization_name)

print(f"Organization name has been written to '{organization_output_path}'.")

# Write the extracted person name to a text file
person_output_filename = "person_name.txt"
person_output_path = os.path.join(person_dir, person_output_filename)
with open(person_output_path, "w") as file:
    file.write(person_name)

print(f"Person name has been written to '{person_output_path}'.")

# Print the extracted names as output
print(f"Extracted Organization Name: {organization_name}")
print(f"Person Name: {person_name}")

Enter the text for extraction: cognizant intuition engineered Tanmay Chakraborty Technical Leadership tanmay.chakraborty@cognizant.com A-4.Nandini Apartment Near Chinar Park tanmay.chakraborty2003@.gmail.com Kolkata-700059India +919748076379
Organization name has been written to '/var/organization/organization_name.txt'.
Person name has been written to '/var/person/person_name.txt'.
Extracted Organization Name: Cognizant
 Person Name: Tanmay Chakraborty


In [26]:
!pip install streamlit
import streamlit as st
import os
import re
import paddleocr
from PIL import Image
import pandas as pd
import spacy
from spacy.training import Example
import random

# Initialize PaddleOCR
ocr_model = paddleocr.PaddleOCR(lang='en')

def extract_organization_and_person_names(text):
    # Extract company name using regex (assuming it appears in the email domain)
    organization_match = re.search(r'@([A-Za-z]+)\.com', text)
    organization_name = organization_match.group(1).capitalize() if organization_match else "Not Found"

    # Extract person name using regex (assuming it appears before the email)
    person_name_match = re.search(r'([A-Za-z\.]+)@', text)
    person_name = person_name_match.group(1).replace('.', ' ').strip().title() if person_name_match else "Not Found"

    return organization_name, person_name

def ocr_image(image):
    result = ocr_model.ocr(image, cls=False)
    text = ""
    for line in result:
        text += ' '.join([word_info[1][0] for word_info in line]) + '\n'
    return text

def main():
    st.title('Organization and Person Name Extractor from Image')
    st.write("Upload an image containing text to extract organization and person names.")

    # File uploader for images
    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None:
        # Display the uploaded image
        image = Image.open(uploaded_file)
        st.image(image, caption='Uploaded Image.', use_column_width=True)
        st.write("")

        st.write("Extracting text from the image...")
        # Perform OCR on the uploaded image
        extracted_text = ocr_image(uploaded_file)

        st.write("Extracted Text:")
        st.write(extracted_text)

        # Extract organization name and person name
        organization_name, person_name = extract_organization_and_person_names(extracted_text)

        # Define the output directory
        output_dir = "output_directory"
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Define the paths for the output text files
        organization_output_path = os.path.join(output_dir, 'organization_name.txt')
        person_output_path = os.path.join(output_dir, 'person_name.txt')

        # Write the extracted names into text files
        with open(organization_output_path, 'w') as f:
            f.write(organization_name)

        with open(person_output_path, 'w') as f:
            f.write(person_name)

        # Display the results
        st.write("### Extracted Information")
        st.write("*Extracted Organization Name:*")
        st.write(organization_name)
        st.write("*Extracted Person Name:*")
        st.write(person_name)

        # Provide download links for the text files
        st.write("### Download Extracted Information")
        st.download_button(label="Download Organization Name", data=open(organization_output_path, 'r').read(), file_name='organization_name.txt')
        st.download_button(label="Download Person Name", data=open(person_output_path, 'r').read(), file_name='person_name.txt')

if __name__ == "__main__":
  main()

[2024/07/27 10:59:58] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

2024-07-27 11:00:00.528 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


In [6]:
import pandas as pd
import spacy
from spacy.training import Example
import random

# Load spaCy model for organization and person names
nlp = spacy.blank("en")

# Define the NER pipeline component for both organization and person names
ner = nlp.add_pipe("ner")
ner.add_label("ORG")
ner.add_label("PERSON")

# Read data from CSV file for organization names
df_org = pd.read_csv("/content/Company Names.csv")  # Update with your CSV file
df_person = pd.read_csv("/content/Indian_Names.csv")  # Update with your CSV file for person names

# Function to check if a value is a valid string
def is_valid_string(value):
    return isinstance(value, str) and len(value) > 0

# Convert data into spaCy training format for organization names
train_examples = []
for _, row in df_org.iterrows():
    text = row["Company Name"]  # Update with the correct column name
    if is_valid_string(text):
        annotations = {"entities": [(0, len(text), "ORG")]}
        example = Example.from_dict(nlp.make_doc(text), annotations)
        train_examples.append(example)

# Convert data into spaCy training format for person names
for _, row in df_person.iterrows():
    text = row["Name"]  # Update with the correct column name
    if is_valid_string(text):
        annotations = {"entities": [(0, len(text), "NAMES")]}
        example = Example.from_dict(nlp.make_doc(text), annotations)
        train_examples.append(example)

# Training loop for both organization and person names
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for iteration in range(50):
        losses = {}
        random.shuffle(train_examples)
        for example in train_examples:
            nlp.update([example], drop=0.5, losses=losses)
        print(f"Iteration {iteration}, Losses: {losses}")

# Save the trained model for both organization and person names
nlp.to_disk("ner_model")


KeyboardInterrupt: 